### Создание парсера для Opencast VM MIEM

Для того, чтобы создать датасет для инференса, необходимо сделать выборку из видео с Opencast. Поскольку все записи длятся в среднем по 1.5 часа, необходимо обрезать эти видео в нескольких местах с наиболее вероятным присутствием людей в кадре. Поскольку все записи на сайте - записи семинаров и лекций, то, исключая 10 минутный перерыв в начале, вероятнее всего увидеть человека в кадре - примерно через час после начала записи.

In [1]:
!pip install --trusted-host pypi.python.org moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 2.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 6.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 6.1 MB/s eta 0:00:0000:0100:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110727 sha256=740b0a6422359380f4f0f1151d2e8d5866ab12cbc03cf865b6a2d8f680bb398b
  Stored in directory: /Users/artem/Library/Caches/pip/wheels/83/b1/d9/119ef7c144b44d591ec0a9a140465133c23ea95d2a161184ba
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


In [2]:
!pip install imageio-ffmpeg

In [3]:
!pip install bs4

In [36]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 4.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled attrs-22.2.0


In [38]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import urllib.request
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

vid_links = []
trim_start = 3600
trim_end = 3610
web_url = 'https://video.miem.hse.ru/engage/ui/index.html'


Сайт Opencast VM MIEM имеет задержку при запросе исходного кода страницы, таким образом, через request получить полный текст html не получится. Необходимо имитировать работу пользователя через selenium.

In [35]:
browser = webdriver.Chrome()
browser.get('https://video.miem.hse.ru/engage/ui/index.html?e=1')

In [39]:
all_txt = browser.page_source

In [40]:
soup = BeautifulSoup(all_txt, "html.parser")
all_vids = soup.findAll('a', class_='tile')
for data in all_vids:
    vid_links.append(data['href'])

print("Number of vids: ", len(vid_links))

Number of vids:  1060


Также после перехода по полученным ссылкам на видео мы оказываемся на страницы плеера, где нужно нажать кнопку play, чтобы получить код нужной нам страницы для скачивания. Здесь нам также помогает selenium.

In [165]:
browser.get('https://video.miem.hse.ru' + vid_links[0])

In [166]:
share = browser.find_element(By.ID, 'lazyLoadThumbnailContainer')
share.click()

In [150]:
soup1 = BeautifulSoup(browser.page_source, "html.parser")
res = soup1.findAll('div', class_='downloadsLinkContainer')
for data in res:
    if 'presenter/delivery' in str(data) and '(854x480@25fps)' in str(data):
        dlink = data.findAll('a', class_='downloadsLinkItem')[0]['href']

print(dlink)

https://video.miem.hse.ru/static/mh_default_org/engage-player/32221332-2729-48c1-9903-a2bdbd0a375a/7ab99570-d519-48c5-8e95-4b2dcb4d5ce0/recording_1679379960_32221332_2729_48c1_9903_a2bdbd0a375a_presenter.mp4


In [151]:
urllib.request.urlretrieve(dlink, 'videoname.mp4')

('videoname.mp4', <http.client.HTTPMessage at 0x16721c4d0>)

Реализуем полный парсер с помощью цикла:

In [41]:
for i in range(1044, len(vid_links)):
    browser.get('https://video.miem.hse.ru' + vid_links[i])
    time.sleep(5)
    share = browser.find_element(By.ID, 'lazyLoadThumbnailContainer')
    share.click()
    time.sleep(7)
    soup1 = BeautifulSoup(browser.page_source, "html.parser")
    res = soup1.findAll('div', class_='downloadsLinkContainer')
    for data in res:
        if 'presenter/delivery' in str(data) and '(854x480@25fps)' in str(data):
            dlink = data.findAll('a', class_='downloadsLinkItem')[0]['href']
    if dlink=='':
        print('SOMETHING WRONG!!!')
        break
    urllib.request.urlretrieve(dlink, 'videoname{}.mp4'.format(i+1))

Поскольку часовые видео нейронка будет обрабатывать слишком долго, необходимо сдедать 20-и секундные сниппеты с видео. Будем выбирать эти кусочки из середины записей, поскольку там наибольшая вероятность встретить хотя бы 1 лицо в кадре:

In [42]:
t1 = 3600
t2 = 3620

for i in range(1044, 1061):
    try:
        ffmpeg_extract_subclip("videoname{}.mp4".format(i), t1, t2, targetname="vid{}_trimmed_20s.mp4".format(i))
    except OSError:
        continue

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command returned an error
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ "

Готово!